# Import Dependencies

In [129]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

from matplotlib import pyplot as plt

from pathlib import Path
from tqdm import tqdm

import warnings

warnings.filterwarnings('ignore')

# Load Data

In [130]:
# load 4 dataframes (returns, nb industries, avg firm size, Sum of BE / Sum of ME)
df = pd.read_csv(str(Path().absolute()) + "/data/48_Industry_Portfolios.CSV") 

# split these dataframes
df_list = np.split(df, df[df.isnull().all(1)].index, axis = 0) 

# clean data and convert date column to index
for i in range(len(df_list)):
    df_list[i] = pd.DataFrame(df_list[i])  
    df_list[i] = df_list[i].dropna()  
    df_list[i].loc[:, "Date"] = df_list[i].loc[:, "Date"].astype("int")  
    df_list[i] = df_list[i].set_index("Date")  
    
    # last data frame has yearly data
    if i == (len(df_list) - 1): 
        df_list[i].index = pd.to_datetime(df_list[i].index, format = "%Y")
        df_list[i].index = df_list[i].index + pd.DateOffset(months = 6)
    else:
        df_list[i].index = pd.to_datetime(df_list[i].index, format = "%Y%m")

# create a dataframe of excess returns, nb of industries and avg sizes
df = df_list[0] / 100
mask = (df <= -0.99)
df[mask] = np.nan

nb_industries = df_list[1]
nb_industries[mask] = np.nan

avg_size = df_list[2]
avg_size[mask] = np.nan

be_over_me = df_list[3]
be_over_me[mask] = np.nan

In [131]:
# market cap of each industry over time
mkt_cap = nb_industries * avg_size
print(mkt_cap.shape)

# book value to market value
be_over_me = be_over_me.resample("1MS").ffill()
print(be_over_me.shape)

# momentum with monthly data
momentum = df.rolling(12).mean()
print(momentum.shape)

(1182, 48)
(1177, 48)
(1182, 48)


# Normalize Data

In [132]:
mkt_cap_ = mkt_cap.loc['1927-06-01':'1973-12-01']
mkt_cap_norm = (mkt_cap_ - mkt_cap_.mean()) / mkt_cap_.std()
print(mkt_cap_norm.shape)

be_over_me_ = be_over_me.loc['1927-06-01':'1973-12-01']
be_over_me_norm = (be_over_me_ - be_over_me_.mean()) / be_over_me_.std()
print(be_over_me_norm.shape)

momentum_ = momentum.loc['1927-06-01':'1973-12-01']
momentum_norm = (momentum_ - momentum_.mean()) / momentum_.std()
print(momentum_norm.shape)

(559, 48)
(559, 48)
(559, 48)


In [133]:
def CRRA(wealth, gamma= 5):
    """"Constant Relative Risk Aversion Utility Function"""

    if gamma == 1:
        return np.log(wealth)
    else:
        return ((1 + wealth) ** (1 - gamma)) / (1 - gamma)


In [134]:
characteristics = np.stack([mkt_cap_norm, be_over_me_norm, momentum_norm], axis= -1)
weights = mkt_cap.iloc[-1]/ mkt_cap.iloc[-1].sum()
theta = np.array([-1.451, 3.606, 1.772])

weights

Agric    0.000859
Food     0.007388
Soda     0.006012
Beer     0.005945
Smoke    0.005225
Toys     0.000644
Fun      0.010471
Books    0.000555
Hshld    0.012793
Clths    0.004255
Hlth     0.005851
MedEq    0.019788
Drugs    0.053875
Chems    0.008270
Rubbr    0.001124
Txtls    0.000225
BldMt    0.005606
Cnstr    0.007072
Steel    0.002262
FabPr    0.000234
Mach     0.022131
ElcEq    0.002505
Autos    0.023701
Aero     0.009449
Ships    0.000854
Guns     0.003317
Gold     0.000960
Mines    0.004196
Coal     0.000311
Oil      0.028253
Util     0.027976
Telcm    0.020845
PerSv    0.003352
BusSv    0.210573
Comps    0.013879
Chips    0.161610
LabEq    0.015992
Paper    0.001822
Boxes    0.001378
Trans    0.019526
Whlsl    0.011183
Rtail    0.085470
Meals    0.014186
Banks    0.059413
Insur    0.034341
RlEst    0.001590
Fin      0.033877
Other    0.028863
Name: 2024-12-01 00:00:00, dtype: float64

# Optimization

In [135]:
def objective(theta:np.ndarray, x:np.ndarray, rets:pd.DataFrame, weights:np.ndarray):
    accrued_wealth = 0
    wealth = 0
    for t in range(x.shape[0]):
        rets_valid = ~pd.isna(rets.iloc[t+1, :])
        x_valid = ~np.isnan(x[t, :, :]).any(axis=1)
        Nt = np.sum(rets_valid & x_valid)
        
        for i in range(x.shape[1]):
            if (np.isnan(rets.iloc[t+1,i])) or (np.isnan(x[t,i,:]).any()):
                continue
            wealth += (weights[i] + theta.T @ x[t,i,:] / Nt) * rets.iloc[t+1,i]
            # print(wealth)
        accrued_wealth += CRRA(wealth)
    
    return - accrued_wealth / x.shape[0]

In [ ]:
init = np.array([-1.398e+01,  4.366e+01, 2.770e+01]) # local solution found
# init = theta
response = minimize(objective, x0= theta, args= (characteristics, df, weights), method= 'BFGS')
response

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 6.504109459241201e-06
        x: [-1.398e+01  4.366e+01  2.770e+01]
      nit: 31
      jac: [ 5.162e-07 -4.060e-06  5.777e-06]
 hess_inv: [[ 6.213e+05 -1.991e+06 -1.280e+06]
            [-1.991e+06  6.377e+06  4.101e+06]
            [-1.280e+06  4.101e+06  2.637e+06]]
     nfev: 132
     njev: 33

In [140]:
next_w = np.zeros(weights.shape)
for i in range(len(weights)):
    next_w[i] = weights[i] + response.x @ characteristics[-1,i,:]

next_w

array([  -7.43295351,  -78.70498608, -124.1852141 , -114.44349746,
        -97.09178763,  -83.28462328, -119.73117457,  -99.03899836,
       -135.01640578,  -71.39234625,   91.55039204, -135.79557445,
       -119.14988839,  -11.18843037,  -77.70046803,  -70.2644453 ,
        -47.89964353,  -89.1861542 ,    5.86978088,  -73.95500159,
        -65.09968707,  -62.66455164,  -37.67263137,   -5.77306234,
        -10.1226922 ,   56.98460499,   21.06145894,  -16.79258927,
        -33.99018355,  -58.70978341,  -47.94499223,  -25.32861034,
        -90.16855713, -110.41425179,  -97.73546587,  -60.89517008,
        -61.49691113,  -36.29734226,  -91.25652982,  -68.31459505,
        -98.63052302, -104.07855996, -137.11044951, -100.39579063,
        -78.98779083,  -97.56633802, -130.54605147, -215.85639281])

In [141]:
next_weights = np.zeros(next_w.shape)
# apply equation 16
for i in range(len(next_w)):
    w_pos = np.clip(next_w, 0, None)
    next_weights[i] = max(0, next_w[i]) / np.sum(w_pos)

next_weights

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.52175503, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.03345248, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.32476108, 0.1200314 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ])

# Out-of-Sample Testing